In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Predict next day rain in Australia...

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">0. Introduction</p>


This notebook purpose is clearly understand to whetherAUS(predicition to whether rainy for  AUS ) dataset. In here mostly used visualation for this. For prediction used deep learning. In conclusion our ACC: %85.88 found.

Description About Columns:

* Date: The date of observation
* Location: The common name of the location of the weather station
* MinTemp: The minimum temperature in degrees celsius
* MaxTemp: The maximum temperature in degrees celsius
* Rainfall: The amount of rainfall recorded for the day in mm
* Evaporation: The so-called Class A pan evaporation (mm) in the 24 hours to 9am
* Sunshine: The number of hours of bright sunshine in the day
* WindGustDir: The direction of the strongest wind gust in the 24 hours to midnight
* WindGustSpeed: The speed (km/h) of the strongest wind gust in the 24 hours to midnight
* WindDir9am: Direction of the wind at 9am
* WindDir3pm: Direction of the wind at 3pm
* WindSpeed9am: Wind speed (km/hr) averaged over 10 minutes prior to 9am
* WindSpeed3pm: Wind speed (km/hr) averaged over 10 minutes prior to 3pm
* Humadity9am: Humidity (percent) at 9am
* Humadity3pm: Humidity (percent) at 3pm
* Pressure9am: Atmospheric pressure (hpa) reduced to mean sea level at 9am
* Pressure3pm: Atmospheric pressure (hpa) reduced to mean sea level at 3pm
* Cloud9am: Fraction of sky obscured by cloud at 9am. This is measured in "oktas", which are a unit of eigths. It records how many eigths of the sky are obscured by cloud. A 0 measure indicates completely clear sky whilst an 8 indicates that it is completely overcast
* Cloud3pm: Fraction of sky obscured by cloud (in "oktas": eighths) at 3pm
* Temp9am: Temperature (degrees C) at 9am
* Temp3pm: Temperature (degrees C) at 3pm
* RainToday: Boolean: 1 if precipitation (mm) in the 24 hours to 9am exceeds 1mm, otherwise 0
* RainTomorrow: The amount of next day rain in mm. Used to create response variable RainTomorrow. A kind of measure of the "risk".

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">1. Imports</p>


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sbn
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout,BatchNormalization,Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

* Read dataset and show first 5 rows.

In [ ]:
data = pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")

In [ ]:
data.head()

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">2. Information About Dataset</p>

This step is we will learn to some information about dataset.
* Info is show us non-null(not empty) and dtype(object,float64 etc.)
* Describe is show us the data description(count,min,max,std,mean etc.)
* Isnull is show us the null values in data.
* Unique is show us the each column have how many different values.
* Corr is show us the each column corelation with target("RainTomorrow") column and we showed on the graph(Only numeric data)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
# shows the numbers of unique values
for col in data.columns:
    n_uniq = data[col].nunique()
    print(f"Column Name:{col}, Unique Count: {n_uniq}")

In [ ]:
# Show the values of  RainToday column
data["RainToday"].value_counts()

In [ ]:
# Correlation amongst numeric attributes
corrmat = data.corr()
plt.subplots(figsize=(16,16))
sbn.heatmap(corrmat,annot=True, square=True)

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">3. Visualation The Dataset</p>

This section we will try to show on graph some of situation. We will create lots of scenario and try to show on the graph so we can understand to dataset clearly.

* Firstly we will look RainToday column values count on graph. 

In [ ]:
sbn.countplot(data = data,x = "RainToday",palette="Dark2")

* Next step, we will look relation MinTemp and Location column. MinTemp column is minimum temperature. So our purpose  has find to minimum temperature cities and we can show on the graph.

In [ ]:
data[["Location","MinTemp"]].groupby("Location").mean().sort_values(by="MinTemp").iloc[:20]

* The next step is show us the above scenario on the graph.

In [ ]:
fig = px.bar(data[["Location","MinTemp"]].groupby("Location").mean().sort_values(by="MinTemp").iloc[:20],x="MinTemp",
            labels={"MinTemp":"Minimum Temperature °C","Location":"Location Names"})
fig.update_layout(
    title={
        "text":"Top 20 Location has minimum temperature",
        "x":0.5,
        "y":0.95,
        "xanchor":"center",
        "yanchor":"top"
})
fig.show()
#Top 20 location has minimum temperature

* Firstly, we will look relation MaxTemp and Location column. MaxTemp column is maximum temperature. So our purpose has find to maximum temperature cities and we can show on the graph.

In [ ]:
data[["Location","MaxTemp"]].groupby("Location").mean().sort_values(ascending=False,by="MaxTemp").iloc[:20]

* The next step is show us the above scenario on the graph.

In [ ]:
fig = px.bar(data[["Location","MaxTemp"]].groupby("Location").mean().sort_values(ascending=False,by="MaxTemp").iloc[:20],x = "MaxTemp",
            labels = {"MaxTemp":"Maximum Temperature °C","Location":"Location Names"})
fig.update_layout(
title={
    "x":0.5,
    "y":0.95,
    "xanchor":"center",
    "yanchor":"top",
    "text": "Top 20 Location has maximium temperature"
})
fig.show()
# Top 20 location temp high 

* Next step, we will look Max Temp relation by Location on the year.Thus we will learn which year has  most high temperature  value and most low temperature value.
* We just will show 12 location other wise they pick  a lot of take up space. For this we take 12 max temperature location.


In [ ]:
# Show of 12 cities with the max temp column by years
data["year"] = data["Date"].apply(lambda x: x.split("-")[0])
data["months"] = data["Date"].apply(lambda x: x.split("-")[1])
data["day"] = data["Date"].apply(lambda x: x.split("-")[2])

data_location = ["Katherine","Darwin","Uluru","Cairns","Townsville","AliceSprings","Moree","Brisbane","PearceRAAF","Cobar","GoldCoast","PerthAirport"]

grp = data.groupby("Location")

num_rows, num_cols = 6,2
f, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(12, 12))
#f.suptitle('Distribution of Features', fontsize=16)

for index, city in enumerate(data_location):
    i,j = (index // num_cols, index % num_cols)
    city_group = grp.get_group(city)
    df=city_group[["MaxTemp","year"]].groupby("year").mean()
    sbn.lineplot(x="year",y="MaxTemp", data=df, ax=axes[i,j],marker="o").set_title(city)

plt.tight_layout()
plt.show()


* Next step, we will look Min Temp relation by Location on the year.Thus we will learn which year has  most low temperature  value and most low temperature value.
* We just will show 12 location other wise they pick  a lot of take up space. For this we take 12 min temperature location.


In [ ]:
data["year"] = data["Date"].apply(lambda x: x.split("-")[0])
data["months"] = data["Date"].apply(lambda x: x.split("-")[1])
data["day"] = data["Date"].apply(lambda x: x.split("-")[2])

data_location = ["MountGinini","Canberra","Tuggeranong","Ballarat","Launceston","Sale","Bendigo","Dartmoor","MountGambier","Nhil","Hobart","SalmonGums"]

grp = data.groupby("Location")

num_rows,num_cols = 6,2
f ,axes = plt.subplots(nrows = num_rows,ncols=num_cols,figsize=(12,12))
for index,city in enumerate(data_location):
    i,j = (index // num_cols , index % num_cols)
    city_group = grp.get_group(city)
    df = city_group[["MinTemp","year"]].groupby("year").mean()
    sbn.lineplot(data = df,x = "year",y="MinTemp",ax = axes[i,j],marker="o").set_title(city)

plt.tight_layout()
plt.show()

* Next step, we will look which Location most rainfall take.

In [ ]:
# mean rainfall all data 
data["Rainfall"].mean()

In [ ]:
# Top 10 most rainfall on location 
data[["Location","Rainfall"]].groupby("Location").mean().sort_values(by="Rainfall",ascending=False).iloc[:10]

In [ ]:
fig = px.bar(data[["Location","Rainfall"]].groupby("Location").mean().sort_values(by="Rainfall",ascending=False).iloc[:10],x="Rainfall",labels={"Location":"Location Names","Rainfall":"Rainfall Amount in day(mm)"})
fig.update_layout(
title={
    "x":0.5,
    "y":0.95,
    "xanchor":"center",
    "yanchor":"top",
    "text": "Top 10 Location with maximium rainfall"
})
fig.show()

* Next step, we will look at the amount of rainfall in Location by years. In here we just selected twelve cities but these Locations are the ones with the highest rainfall.

In [ ]:
data["year"] = data["Date"].apply(lambda x: x.split("-")[0])
data["months"] = data["Date"].apply(lambda x: x.split("-")[1])
data["day"] = data["Date"].apply(lambda x: x.split("-")[2])

grp = data.groupby("Location")
data_location = ["Newcastle","Katherine","MountGinini","Sydney","NorahHead","Townsville","Williamtown","Wollongong","GoldCoast","CoffsHarbour","Darwin","Cairns"]

num_cols,num_rows = 2,6
f,axes = plt.subplots(nrows=num_rows,ncols=num_cols,figsize=(12,12))

for index,city in enumerate(data_location):
    i,j = (index//num_cols,index % num_cols)
    city_group = grp.get_group(city)
    df = city_group[["Rainfall","year"]].groupby("year").mean()
    sbn.lineplot(data=df,ax = axes[i,j],marker="o").set_title(city)
plt.tight_layout()
plt.show()

* Next step the amount of sunlight received in a day by Location (hours)

In [ ]:
fig = px.bar(data.groupby("Location").mean().sort_values(ascending = False,by ="Sunshine").iloc[:10],x="Sunshine",labels={"Sunshine":"Sunshine (hours)","Location":"Location Names"})
fig.update_layout(
title={
    "x":0.5,
    "y":0.95,
    "xanchor":"center",
    "yanchor":"top",
    "text": "Top 10 Location with maximium Sunlight in a day"
})
fig.show()


* Next step the amount of sunlight received in a per months by Location (hours) 

In [ ]:
data[["Location","Sunshine"]].groupby("Location").mean().sort_values(ascending = False,by ="Sunshine").iloc[:12]

In [ ]:
data["year"] = data["Date"].apply(lambda x: x.split("-")[0])
data["months"] = data["Date"].apply(lambda x: x.split("-")[1])
data["day"] = data["Date"].apply(lambda x: x.split("-")[2])
grp = data.groupby("Location")
data_location = ["AliceSprings","Woomera","Moree","PerthAirport","PearceRAAF","Perth","Cobar","Darwin","Mildura","Townsville","WaggaWagga","Brisbane"]

num_cols ,num_rows = 2,6

f,axes = plt.subplots(nrows = num_rows,ncols =num_cols,figsize=(12,12))

for index,city in enumerate(data_location):
    i,j = (index//num_cols,index % num_cols)
    city_group = grp.get_group(city)
    df = city_group[["Sunshine","months"]].groupby("months").mean()
    sbn.lineplot(data=df,ax = axes[i,j],marker="o").set_title(city)
plt.tight_layout()
plt.show()

* Next step is will be show us the most windy Location.

In [ ]:
data[["Location","WindGustSpeed"]].groupby("Location").mean().sort_values(ascending=False,by="WindGustSpeed").iloc[:10]

In [ ]:
fig = px.bar(data[["Location","WindGustSpeed"]].groupby("Location").mean().sort_values(ascending=False,by="WindGustSpeed").iloc[:10],x="WindGustSpeed",
                labels={"Location":"Location Names","WindGustSpeed":"Wind Speed (km/h)"})
fig.update_layout(
title={
    "x":0.5,
    "y":0.95,
    "xanchor":"center",
    "yanchor":"top",
    "text": "Top 10 Location with maximum Windy"
})
fig.show()

* Next step, we will show top 12 most windy Location by months mean

In [ ]:
data["year"] = data["Date"].apply(lambda x: x.split("-")[0])
data["months"] = data["Date"].apply(lambda x: x.split("-")[1])
data["day"] = data["Date"].apply(lambda x: x.split("-")[2])

grp = data.groupby("Location")
data_location = ["Hobart","MelbourneAirport","Wollongong","Ballarat","Woomera","MountGambier","Nhil","GoldCoast","Portland","Williamtown","Nuriootpa","AliceSprings"]

num_cols, num_rows = 2,6
f,axes = plt.subplots(nrows= num_rows,ncols = num_cols,figsize=(12,12))

for index,city in enumerate(data_location):
    i,j = (index // num_cols,index % num_cols)
    city_group =grp.get_group(city)
    df=city_group[["WindGustSpeed","months"]].groupby("months").mean()
    sbn.lineplot(data=df,ax = axes[i,j],marker="o").set_title(city)
plt.tight_layout()
plt.show()

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">4. Preprocessing</p>

This section we will preprocessing for training. Firstly we will delete some columns, actually we don't need these columns. 
Some of them (year,day,months) we added  we added these for visualation but know we need to delete them. Also we will delete Date column because it's not meaning for the training every row it has different value and it's can effect badly our learning algorithm(deep learning). After all these we need to clean the data, our dataset has lots of NaN value and we need to decide what can we do for these. For RainToday and RainTomorrow columns i decided the delete NaN values because RainTomorrow is our target column and it's can't be NaN. F For the others columns NaN values i decided to fill each with column mean. Some columns had categorical values and our deep learning algorithm can't understand these so we encode these. For this  i used the LabelEncoder these columns (WindDir3pm,RainToday,Windir9am,WindGustDir,Location). After all of them dataset ready for training. 

In [ ]:
data["RainTomorrow"].isnull().sum()

* Data Cleaning to some NaN values

In [ ]:
data = data.drop(["year","day","months"],axis=1)
data.drop(["Date"],axis=1,inplace =True)
data = data.dropna(how ="all")
data = data.dropna(subset = ["RainToday","RainTomorrow"],how="any")
data = data.dropna(subset= ["WindDir9am","WindGustDir","WindDir3pm"],how="any")

* Fill NaN values

In [ ]:
def fill_mean(column):
    for col in column:
        data[col] = data[col].fillna(data[col].mean())
columns_fill = data.columns.drop(["RainToday","RainTomorrow","WindDir9am","WindDir3pm","WindGustDir","Location"])
fill_mean(columns_fill)

* Encode the categorical columns

In [ ]:
def preprocessing(column):
    lbl = LabelEncoder()
    for col in column:
        lbl.fit(data[col])
        data[col] = lbl.transform(data[col].values.reshape(-1,))
columns_process = ["WindDir3pm","RainToday","WindDir9am","WindGustDir","Location"]
preprocessing(columns_process)

* Decide train and test data columns

In [ ]:
feature_col = data.columns.drop("RainTomorrow")
train = data[feature_col]
target = data["RainTomorrow"].map({"Yes":1,"No":0}) #In here encode(0,1) target because i will use regression. 

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">5. Train And Test Split </p>

In this section we will split the dataset into training and testing also we will split the training data set that we reserved for training so we to obtain validation dataset. Thus we can know,  what it's can do for real dataset. 

In [ ]:
x_tr,x_test,y_tr,y_test = train_test_split(train,target,test_size=0.2,random_state=1)
x_train,x_val,y_train,y_val = train_test_split(x_tr,y_tr,test_size=0.1,random_state=1)

In [ ]:
print("X train shape",x_train.shape)
print("Y train shape",y_train.shape)
print("X Val shape",x_val.shape)
print("Y Val shape",y_val.shape)

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">6. Scale Data </p>

In this section we will scale our training and testing dataset thus deep learning algorithm can be better. This is be happen: standart scaler is standardize features by removing the mean and scaling to unit variance, so deep learning algorithm can calculate easyly and it's can be learn better.

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)
X_val = sc.transform(x_val)

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">7. Create Deep Learning Model </p>

In this section, we will create our deep learning model. Firstly we need to know input_dim this is our feature columns(feature_col) count. After we will decide to how much Dense we need time we try for this mostly. In here isn't have special number, like you can start with this number. Mostly people start with input_dim number. They divide input_dim number to 2 and this keep going to 1. For regression last step just can 1 Dense.

In [ ]:
model = Sequential()

model.add(Dense(25,activation="relu",kernel_initializer="normal",input_dim=21))
model.add(Dense(15,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation="sigmoid"))
model.summary()

* Compile and train the model.

In [ ]:
model.compile(optimizer="adam",loss = "binary_crossentropy",metrics = ["accuracy"])
call_backs = EarlyStopping(monitor = "val_loss",patience = 10, mode = min,restore_best_weights=True )
hist = model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,verbose = 1, epochs = 30,callbacks = [call_backs])

# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">8. Evaluate The Model </p>

In this section we will evaluate the model. For this we will use classification report,confusion matrix and accuracy score. All of this will show us succes the model.

* Next step is show us training loss,validation loss and training accuracy, validation accuracy values for model.

In [ ]:
plt.figure(figsize=(14,3))
plt.subplot(1, 2, 1)
plt.suptitle('Train', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(hist.history['loss'], color ='r', label='Training Loss')
plt.plot(hist.history['val_loss'], color ='b', label='Validation Loss')
plt.legend(loc='upper right')


plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(hist.history['accuracy'], color ='g', label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], color ='m', label='Validation Accuracy')
plt.legend(loc='lower right')

plt.show()

Evaluate Test Data
* Next step, predict test  and compare with test. For this use X_test data.

In [ ]:
#Confusion Matrix
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_pred,y_test)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Accuracy Score
acc = accuracy_score(y_test,y_pred)
print("Accuracy Score: ",acc)

In [ ]:
# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

* Show on the graph to confusion matrix

In [ ]:
sbn.heatmap(cm, annot=True,fmt='.5g',linewidths=5,cmap="Blues",xticklabels='01',
    yticklabels='01')

Evaluate Validation Data
* Next step, predict validation  and compare with y_val. For this use X_val data.

In [ ]:
# Confusion Matrix
y_pred_val = model.predict(X_val)
y_pred_val = (y_pred_val > 0.5)
cm_val = confusion_matrix(y_pred_val,y_val)
print(cm_val)

In [ ]:
# Accuracy Score
acc_val = accuracy_score(y_pred_val,y_val)
print("Accuracy Validation: ",acc_val)

In [ ]:
# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_pred_val, y_val))

* Show on the graph to confusion matrix

In [ ]:
sbn.heatmap(cm_val, annot=True,fmt='.5g',linewidths=5,cmap="Blues",xticklabels='01',
    yticklabels='01')